# 1. Dependencias e setup

In [ ]:
pip install tensorflow opencv-python matplotlib

In [ ]:
import tensorflow as tf
import os

In [ ]:
# Prevenção de erros OUT OF MEMORY
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)

# 2. Remover imagens incompatíveis

In [ ]:
import cv2
import imghdr
data_dir = 'data'
os.listdir(data_dir)

In [ ]:
image_exts = ['jpeg'] # extensões de imagem a serem preservadas

In [ ]:
for image_class in os.listdir(data_dir):
    for image in os.listdir(os.path.join(data_dir, image_class)):
        image_path = os.path.join(data_dir, image_class, image)
        try:
            img = cv2.imread(image_path)
            tip = imghdr.what(image_path)
            if tip not in image_exts:
                print('Image not in ext list {}'.format(image_path))
                os.remove(image_path)
        except Exception as e:
            print('Issue with image {}'.format(image_path))
            # os.remove(image_path)

# 3. Carregar dados

In [ ]:
import numpy as np
from matplotlib import pyplot as plt

In [ ]:
data = tf.keras.utils.image_dataset_from_directory('data')  # Cria um conjunto de dados de imagens a partir do diretório 'data'

In [ ]:
data

In [ ]:
iterador_dataset = data.as_numpy_iterator() # Transformando dataset em objeto iteravel

In [ ]:
batch = iterador_dataset.next() # Obter próximo batch do iterador

In [ ]:
batch # Grupo de imagens, 32 imagens por grupo

In [ ]:
batch[0].shape # Tamanho do batch, dimensões das imagens e seus canais de cores

In [ ]:
batch[1] # Resultados fornecidos das imagens. 0 = Defeito, 1 = Perfeito

In [ ]:
figura, eixo = plt.subplots(ncols=4, figsize=(20,20)) # Criando uma figura e um conjunto de eixos
for indice, img in enumerate(batch[0][:4]): # Iterando sobre 4 imagens no lote
    eixo[indice].imshow(img.astype(int)) # Mostrando a imagem no eixo atual
    eixo[indice].title.set_text(batch[1][indice]) # Definindo o texto do título do eixo com o rótulo correspondente


# 4. Processando imagens

In [ ]:
data = data.map(lambda x,y: (x/255, y)) # Normalizando os valores dos pixels para numeros entre entre 0 e 1

In [ ]:
iterador_normalizados = data.as_numpy_iterator() # Conversão de dataset normalizado para iterável 

In [ ]:
batch = iterador_normalizados.next() # Obter próximo batch do iterável do dataset normalizado

In [ ]:
batch # Mostrando imagens com valores de canais de cores normalizados.

# 5. Separar dados em grupo de treino, validação e teste

In [ ]:
len(data)

In [ ]:
train_size = int(len(data)*.7) #tamanho da divisão de treinamento (70% dos dados)
val_size = int(len(data)*.2) #tamanho da divisão de validação (20% dos dados)
test_size = int(len(data)*.1) #tamanho da divisão de teste (10% dos dados)


In [ ]:
test_size

In [ ]:
train = data.take(train_size) #conjunto de treinamento usando os primeiros exemplos do conjunto de dados original
val = data.skip(train_size).take(val_size) # conjunto de validação usando os exemplos após o conjunto de treinamento e pegando os próximos 'val_size' exemplos
test = data.skip(train_size+val_size).take(test_size) #conjunto de teste usando os exemplos após os conjuntos de treinamento e validação e pegando os próximos exemplos



# 6. Modelo de Deep Learning

In [ ]:
train

In [ ]:
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout
from tensorflow.python.keras.activations import sigmoid

In [ ]:
model = Sequential()

In [ ]:
model.add(Conv2D(16, (3,3), 1, activation='relu', input_shape=(256,256,3)))  # extrair características iniciais da imagem
model.add(MaxPooling2D()) # reduzir a dimensionalidade e preservar características mais importantes
model.add(Conv2D(32, (3,3), 1, activation='relu')) #  capturar características mais complexas
model.add(MaxPooling2D()) # continuar reduzindo a dimensionalidade
model.add(Conv2D(16, (3,3), 1, activation='relu'))#  refinamento final das características
model.add(MaxPooling2D())   # redução final da dimensionalidade
model.add(Flatten()) # Transformar a saída da camada convolucional em um vetor unidimensional
model.add(Dense(256, activation='relu')) # Camada densa (totalmente conectada) para aprendizado de padrões complexos
model.add(Dense(1, activation='sigmoid')) # Camada densa com ativação sigmoidal para classificação binária

In [ ]:
model.compile('adam', loss=tf.losses.BinaryCrossentropy(), metrics=['accuracy'])

# 'adam' é o otimizador que ajusta os pesos do modelo para melhorar o desempenho.

# A função 'BinaryCrossentropy()' é usada para calcular o quão erradas estão as previsões do modelo
# quando temos apenas duas opções possíveis, como "sim" ou "não". Ela compara as previsões do modelo
# com as respostas reais e mede o quão diferente elas estão. Isso ajuda o modelo a aprender a acertar mais.

# 'accuracy' é a métrica que será monitorada durante o treinamento, medindo a precisão das previsões do modelo.

In [ ]:
model.summary()

# A função model.summary() é usada para exibir um resumo detalhado da arquitetura do modelo de rede neural.
# Ele mostra informações sobre cada camada da rede, incluindo o nome da camada, o tipo de camada,
# o formato de saída (shape) da camada e o número total de parâmetros treináveis na rede.

# 7. Treinando

In [ ]:
logdir='logs'

In [ ]:
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=logdir)

In [ ]:
hist = model.fit(train, epochs=20, validation_data=val, callbacks=[tensorboard_callback])

# A função model.fit() é usada para treinar o modelo de rede neural.
# 'train' são os dados de treinamento que contêm exemplos e rótulos para ensinar o modelo.
# 'epochs' define quantas vezes o modelo verá todo o conjunto de treinamento durante o treinamento.
# 'validation_data' é usado para avaliar o modelo em um conjunto de validação durante o treinamento,
# para entender como ele está se saindo em dados que não foram usados no treinamento.
# 'callbacks' permite adicionar funcionalidades extras durante o treinamento, como o uso do TensorBoard
# para visualizar o progresso do treinamento em tempo real.

# 8. Mostrar resultados do treinamento

In [ ]:
fig = plt.figure()
plt.plot(hist.history['loss'], color='teal', label='loss')  # Plota a curva de perda do treinamento ('loss') em cor teal
plt.plot(hist.history['val_loss'], color='orange', label='val_loss')  # Plota a curva de perda da validação ('val_loss') em cor laranja
fig.suptitle('Loss', fontsize=20)  # Adiciona um título à figura para indicar a curva de perda
plt.legend(loc="upper left")  # Adiciona uma legenda na parte superior esquerda
plt.show()  # Exibe o gráfico


In [ ]:
fig = plt.figure()
plt.plot(hist.history['accuracy'], color='teal', label='accuracy')  # Plota a curva de precisão do treinamento ('accuracy') em cor teal
plt.plot(hist.history['val_accuracy'], color='orange', label='val_accuracy')  # Plota a curva de precisão da validação ('val_accuracy') em cor laranja
fig.suptitle('Accuracy', fontsize=20)  # Adiciona um título à figura para indicar a curva de precisão
plt.legend(loc="upper left")  # Adiciona uma legenda na parte superior esquerda
plt.show()  # Exibe o gráfico


# 9. Avaliar

In [ ]:
from tensorflow.python.keras.metrics import Precision, Recall, BinaryAccuracy

In [ ]:
pre = Precision()  # Cria um objeto de métrica para calcular a precisão
re = Recall()  # Cria um objeto de métrica para calcular o recall
acc = BinaryAccuracy()  # Cria um objeto de métrica para calcular a acurácia binária

In [ ]:
len(test)

In [ ]:
# Loop pelos batches dos dados de teste.
for batch in test.as_numpy_iterator():
    item, label = batch # Obtém um exemplo e seu rótulo do batch
    resultado = loaded_model.predict(item) # Faz uma previsão usando o modelo carregado
    pre.update_state(label, resultado) # Atualiza a métrica de precisão com os resultados da previsão
    re.update_state(label, resultado) # Atualiza a métrica de recall com os resultados da previsão
    acc.update_state(label, resultado) # Atualiza a métrica de acurácia binária com os resultados da previsão

In [ ]:
print(pre.result(), re.result(), acc.result()) # Imprime os resultados das métricas calculadas.

# 10. Testar modelo

In [ ]:
import cv2, random, os
import numpy as np
import tensorflow as tf
from matplotlib import pyplot as plt

from tensorflow.python.keras.models import load_model
modelo_carregado = load_model("models/pecasdefeituosas.keras") # Carrega o modelo previamente treinado a partir do arquivo

In [ ]:
diretorio_sem_defeito = 'test/okay/'
diretorio_com_defeito = 'test/defect/'
imagens_sem_defeito = os.listdir(diretorio_sem_defeito)
imagens_com_defeito = os.listdir(diretorio_com_defeito)

### Escolher imagem aleatória (sem e com defeito)

In [ ]:
img = random.choice(imagens_sem_defeito)
img = cv2.imread(os.path.join(diretorio_sem_defeito, img))

#img = random.choice(imagens_com_defeito)
#img = cv2.imread(os.path.join(diretorio_com_defeito, img))

### Exibindo imagem selecionada aleatóriamente

In [ ]:
img = tf.image.resize(img, (256,256)) #trocar tamanho para o tamanho adequado
plt.imshow(img.numpy().astype(int))
plt.show()
plt.show()

### Normalizando e prevendo com base na imagem selecionada aleatóriamente

In [ ]:
array_predicao = modelo_carregado.predict(np.expand_dims(img/255, 0))
array_predicao

### Tratamento de resultado e formatação

In [ ]:
lista_predicao = array_predicao.astype(float).tolist()
valor_predicao = lista_predicao[0][0]
valor_predicao_texto = format(valor_predicao, ',.8f') # Formata o valor de previsão para exibir com 8 casas decimais
print("Valor predição sem formatação: ", valor_predicao)
print("Valor predição formatado: ",valor_predicao_texto)

### Exibição da predição

In [ ]:
if valor_predicao > 0.9:
    print(f'Peça sem defeitos')  # Se o valor de previsão for maior que 0.9, imprime "Peça sem defeitos"
else:
    print(f'Peça com defeitos')  # Caso contrário, imprime "Peça com defeitos"

# 11. Salvando modelo

In [ ]:
model.save('/models/pecasdefeituosas.keras')  # Salva o modelo treinado em um arquivo chamado 'pecasdefeituosas.keras'
